In [54]:
# -*- coding:utf8 -*-
import os
import csv
import pandas as pd
import time
import gc
path_train = "~/data/dm/train.csv"  # 训练文件
path_test = "~/data/dm/test.csv"  # 测试文件
path_test_out = "model/"
import pandas as pd
import numpy as np
print('building model started..')
from xgboost import XGBRegressor

building model started..


In [55]:
train_data = pd.read_csv(path_train)
train_data.head()
input_label =["TERMINALNO","TRIP_ID", "DIRECTION", "HEIGHT"]
output_label = ['Y']
train_input = train_data[input_label]    
train_output = train_data[output_label]

datanum = len(train_input)
print('length of train_data : ', datanum)

train_input1 = train_input.drop(0)

train_input1.loc[datanum] = train_input1.loc[1]

train_input1 = train_input1.reset_index(drop=True)

train_input = train_input1 - train_input

print('operate sub is done')
train_input = train_input[(train_input.TERMINALNO==0)&(train_input.TRIP_ID==0)]
train_input = train_input.abs()

train_input['Y'] = train_output['Y']
train_input.head(10)

length of train_data :  53284
operate sub is done


,TERMINALNO,TRIP_ID,DIRECTION,HEIGHT,Y
0,0.0,0.0,12.0,0.091431,0.0
1,0.0,0.0,50.0,5.132202,0.0
2,0.0,0.0,41.0,3.586426,0.0
3,0.0,0.0,36.0,1.716187,0.0
6,0.0,0.0,106.0,13.079101,0.0
7,0.0,0.0,24.0,25.451294,0.0
8,0.0,0.0,83.0,21.666260,0.0
9,0.0,0.0,265.0,33.920410,0.0
10,0.0,0.0,11.0,6.058960,0.0
11,0.0,0.0,84.0,4.577759,0.0


In [59]:
start = time.time()

reg = XGBRegressor(max_depth=5,min_child_weigh=5,
                    eta=0.025,gamma=0.06,
                    subsample=1,learning_rate=0.01, 
                    n_estimators=40,silent=True, 
                    n_jobs=-1,objective='reg:linear')
reg.fit(train_input[["DIRECTION", "HEIGHT"]], train_input[output_label])

end = time.time()
time_elapsed = end - start
print('training data in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

training data in 0m 0s


In [60]:
test_data = pd.read_csv(path_test)
pre_input = test_data[input_label]

start = time.time()  
pre_input1 = pre_input.drop(0)

pre_input1.loc[datanum] = pre_input1.loc[1]

pre_input1 = pre_input1.reset_index(drop=True)

pre_input = pre_input1 - pre_input

pre_input = pre_input.abs()

a = pre_input[(pre_input.TERMINALNO!=0)].index.tolist()


In [61]:
y_pred = reg.predict(pre_input[["DIRECTION", "HEIGHT"]])
print(y_pred.size)

pre_input['Y']=y_pred
with (open(os.path.join(path_test_out, "test.csv"), mode="w")) as outer:
    writer = csv.writer(outer)
    writer.writerow(["Id", "Pred"])
    for i in range(len(a)):
        if i==0:
            part = pre_input[0:a[i]]           
        else:
            part = pre_input[a[i-1]:a[i]]
        avg = (part[(part.TERMINALNO==0)&(part.TRIP_ID==0)]['Y'].min()
            +part[(part.TERMINALNO==0)&(part.TRIP_ID==0)]['Y'].mean())*0.5
        writer.writerow([i+1, avg])

16022
